In [1]:
import time
from datetime import datetime
from os import mkdir
from os.path import exists, join

import pandas as pd
import numpy as np
import scipy as sp

import psutil

In [2]:
if not exists('logs'):
    mkdir(log_dir)

In [3]:
now = datetime.now()
log_dir = join('logs', now.strftime('%d-%H'))
if not exists(log_dir):
    mkdir(log_dir)
log_dir

'logs/01-20'

In [4]:
timespans = [str(x.date())[-2:] for x in pd.date_range('2017/6/5', '2017/6/11')]

In [5]:
worker_dirs = timespans + ['test']

In [6]:
feture_extract_path = ['extract_hotel_feature', 'extract_basic_room_feature',
                      'extract_hotel_room_feature', 'extract_room_feature',
                      'extract_uid_feture', 'extract_order_feature']

In [7]:
# feture_extract_path = ['extract_order_feature']

In [8]:
feture_combine_path = ['combine_sample']

In [9]:
max_run = 4

In [10]:
def get_args(feture_extract_path, worker_dirs):
    for p in feture_extract_path:
        for d in worker_dirs:
            yield [p, d]

In [11]:
def check(p):
    pm = p.memory_info()
    is_zero = all(map(lambda x:getattr(pm, x)==0, ['rss','vms', 'shared', 'text', 'lib', 'data', 'dirty']))
    return is_zero

In [12]:
def is_wait(pool, max_pool=4):
    if len(pool) < max_pool:
        print(datetime.now(), 'pool size', len(pool))
        return False
    else:
        time.sleep(10)
        not_run_p = []
        for i, p in enumerate(pool[:]):
            not_run = check(p)
            print(datetime.now(), 'process', i, 'not runing' if not_run else 'run')
            if not_run:
                not_run_p.append(i)
        not_run_p.sort(reverse=True)
        for i in not_run_p:
            p = pool.pop(i)
            p.kill()
        return True

In [13]:
def run_args(extract_args, max_pool=4):
    pool = []
    for p, n in extract_args:
        print(datetime.now(), 'begin', p, n)
        fn = join(log_dir, '{}_{}.txt'.format(p, n))
        pool.append(psutil.Popen(['python', p+'.py', n], stdout=open(fn, 'w')))
        while is_wait(pool, max_pool):
            print(datetime.now(), 'run in ', p, n)
    while is_wait(pool, 1):
        print('wait for pool over')

In [14]:
# for p, n in extract_args:
#     print(datetime.now(), 'begin', p, n)
#     fn = join(log_dir, '{}_{}.txt'.format(p, n))
#     pool.append(psutil.Popen(['python', p+'.py', n], stdout=open(fn, 'w')))
#     while is_wait(pool):
#         print(datetime.now(), 'run in ', p, n)

In [15]:
extract_args = get_args(feture_extract_path, worker_dirs)
combine_args = get_args(feture_combine_path, worker_dirs)

In [16]:
run_args(extract_args, max_run)

2017-08-01 20:44:43.818346 begin extract_hotel_feature 05
2017-08-01 20:44:43.825060 pool size 1
2017-08-01 20:44:43.825165 begin extract_hotel_feature 06
2017-08-01 20:44:43.830011 pool size 2
2017-08-01 20:44:43.830129 begin extract_hotel_feature 07
2017-08-01 20:44:43.835656 pool size 3
2017-08-01 20:44:43.835732 begin extract_hotel_feature 08
2017-08-01 20:45:13.872013 process 0 not runing
2017-08-01 20:45:13.872935 process 1 not runing
2017-08-01 20:45:13.873222 process 2 not runing
2017-08-01 20:45:13.873447 process 3 not runing
2017-08-01 20:45:13.875094 run in  extract_hotel_feature 08
2017-08-01 20:45:13.875192 pool size 0
2017-08-01 20:45:13.875256 begin extract_hotel_feature 09
2017-08-01 20:45:13.881016 pool size 1
2017-08-01 20:45:13.881111 begin extract_hotel_feature 10
2017-08-01 20:45:13.886034 pool size 2
2017-08-01 20:45:13.886130 begin extract_hotel_feature 11
2017-08-01 20:45:13.890634 pool size 3
2017-08-01 20:45:13.890746 begin extract_hotel_feature test
2017-08-0

In [17]:
run_args(combine_args, max_run)

2017-08-01 20:51:44.517065 begin combine_sample 05
2017-08-01 20:51:44.523457 pool size 1
2017-08-01 20:51:44.523581 begin combine_sample 06
2017-08-01 20:51:44.528743 pool size 2
2017-08-01 20:51:44.528841 begin combine_sample 07
2017-08-01 20:51:44.533807 pool size 3
2017-08-01 20:51:44.533909 begin combine_sample 08
2017-08-01 20:52:14.557395 process 0 not runing
2017-08-01 20:52:14.558207 process 1 not runing
2017-08-01 20:52:14.558470 process 2 not runing
2017-08-01 20:52:14.558700 process 3 not runing
2017-08-01 20:52:14.560243 run in  combine_sample 08
2017-08-01 20:52:14.560340 pool size 0
2017-08-01 20:52:14.560403 begin combine_sample 09
2017-08-01 20:52:14.567198 pool size 1
2017-08-01 20:52:14.567378 begin combine_sample 10
2017-08-01 20:52:14.575662 pool size 2
2017-08-01 20:52:14.575794 begin combine_sample 11
2017-08-01 20:52:14.581675 pool size 3
2017-08-01 20:52:14.581773 begin combine_sample test
2017-08-01 20:52:44.620889 process 0 not runing
2017-08-01 20:52:44.6218